In [1]:
import numpy as np
import simpy
import random

In [2]:
conveyor_length = 10

In [3]:
test_conveyort = np.zeros(conveyor_length)

In [4]:
test_conveyort

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])

In [5]:
import simpy
import yaml
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from collections import deque

# -------------- CONFIGURATION READER --------------
def load_config(path="config.yaml"):
    with open(path, "r") as f:
        return yaml.safe_load(f)

# -------------- MODULAR COMPONENTS --------------

class ConveyorBelt:
    def __init__(self, env, name, capacity, speed):
        self.env = env
        self.name = name
        self.capacity = capacity
        self.speed = speed
        self.store = simpy.Store(env, capacity=capacity)
        self.in_transit = deque(maxlen=capacity)
        self.history = []

    def put(self, item):
        self.in_transit.append(self.env.now)
        return self.store.put(item)

    def get(self):
        yield self.env.timeout(1.0 / self.speed)
        self.in_transit.popleft()
        item = yield self.store.get()
        return item

    def utilization(self):
        return len(self.store.items) / self.capacity

    def is_bottleneck(self):
        return len(self.store.items) >= self.capacity

class Buffer:
    def __init__(self, env, name, capacity):
        self.env = env
        self.name = name
        self.capacity = capacity
        self.store = simpy.Store(env, capacity=capacity)

    def put(self, item):
        return self.store.put(item)

    def get(self):
        item = yield self.store.get()
        return item

    def utilization(self):
        return len(self.store.items) / self.capacity

    def is_bottleneck(self):
        return len(self.store.items) >= self.capacity

class Machine:
    def __init__(self, env, name, capacity, speed):
        self.env = env
        self.name = name
        self.capacity = capacity
        self.speed = speed
        self.busy = False

    def process(self, input_store, output_store):
        while True:
            self.busy = True
            items = []
            for _ in range(self.capacity):
                try:
                    item = yield input_store.get()
                    items.append(item)
                except simpy.exceptions.Empty:
                    break
            if items:
                yield self.env.timeout(1.0 / self.speed)
                for item in items:
                    yield output_store.put(item)
            else:
                yield self.env.timeout(0.1)
            self.busy = False

class Storage:
    def __init__(self, env, capacity):
        self.env = env
        self.capacity = capacity
        self.store = simpy.Store(env, capacity=capacity)

    def put(self, item):
        return self.store.put(item)

    def get(self):
        item = yield self.store.get()
        return item

# -------------- SIMULATION ENVIRONMENT --------------

class AssemblyLine:
    def __init__(self, config):
        self.env = simpy.Environment()
        self.config = config
        # Funnel (acts as the initial source)
        self.funnel = simpy.Store(self.env, capacity=config['funnel']['capacity'])
        for i in range(config['target_bottles']):
            self.funnel.put(f"raw_material_{i}")

        # Conveyor belts
        self.conv = {k: ConveyorBelt(self.env, k, **v)
                     for k, v in config['conveyor_belts'].items()}
        # Buffers
        self.buffer = {k: Buffer(self.env, k, **v)
                       for k, v in config['buffers'].items()}
        # Machines
        self.machines = {
            'blow':  Machine(self.env, 'blow',  config['machines']['blow']['capacity'], config['machines']['blow']['speed']),
            'clean': Machine(self.env, 'clean', config['machines']['clean']['capacity'], config['machines']['clean']['speed']),
            'wrap':  Machine(self.env, 'wrap',  config['machines']['wrap']['capacity'], config['machines']['wrap']['speed'])
        }
        # Storage, robotic arm, platform
        self.storage = Storage(self.env, config['storage']['capacity'])
        self.platform = simpy.Store(self.env, capacity=config['platform']['capacity'])

        # History for visualization
        self.history = {name: [] for name in
            ['funnel', *self.conv.keys(), *self.buffer.keys(), 'storage', 'platform']}

    def run(self):
        env = self.env
        config = self.config

        def source():
            while True:
                if len(self.funnel.items) > 0:
                    item = yield self.funnel.get()
                    yield self.conv['conv1'].put(item)
                else:
                    break
                yield env.timeout(0.01) # slight pause to avoid instant transfer

        def transfer(name, from_store, to_store):
            while True:
                item = yield from_store.get()
                yield to_store.put(item)
                yield env.timeout(0.01)

        # Set up processes for each stage in the assembly line
        env.process(source())
        env.process(transfer('conv1->blow', self.conv['conv1'].store, self.conv['conv1'].store)) # For smoothness
        env.process(self.machines['blow'].process(self.conv['conv1'].store, self.conv['conv2'].store))
        env.process(transfer('conv2->buffer1', self.conv['conv2'].store, self.buffer['buffer1'].store))
        env.process(transfer('buffer1->conv3', self.buffer['buffer1'].store, self.conv['conv3'].store))
        env.process(self.machines['clean'].process(self.conv['conv3'].store, self.conv['conv4'].store))
        env.process(transfer('conv4->buffer2', self.conv['conv4'].store, self.buffer['buffer2'].store))
        env.process(transfer('buffer2->conv5', self.buffer['buffer2'].store, self.conv['conv5'].store))
        env.process(self.machines['wrap'].process(self.conv['conv5'].store, self.conv['conv6'].store))
        env.process(transfer('conv6->storage', self.conv['conv6'].store, self.storage.store))
        env.process(transfer('storage->platform', self.storage.store, self.platform))

        # For visualization
        def record_history():
            while True:
                self.history['funnel'].append(len(self.funnel.items))
                for name, c in self.conv.items():
                    self.history[name].append(len(c.store.items))
                for name, b in self.buffer.items():
                    self.history[name].append(len(b.store.items))
                self.history['storage'].append(len(self.storage.store.items))
                self.history['platform'].append(len(self.platform.items))
                yield env.timeout(1)
        env.process(record_history())

    def simulate(self, until=100):
        self.run()
        self.env.run(until=until)

    def get_history(self):
        return self.history

# -------------- VISUALIZATION --------------

def animate_buffers(history, config):
    stages = list(history.keys())
    fig, ax = plt.subplots(figsize=(12,6))
    bars = ax.bar(stages, [0]*len(stages), color='b')

    def update(i):
        for idx, stage in enumerate(stages):
            val = history[stage][i]
            cap = config['funnel']['capacity'] if stage == 'funnel' else \
                  config['platform']['capacity'] if stage == 'platform' else \
                  config['storage']['capacity'] if stage == 'storage' else \
                  config['buffers'][stage]['capacity'] if stage in config['buffers'] else \
                  config['conveyor_belts'][stage]['capacity'] if stage in config['conveyor_belts'] else 1
            bars[idx].set_height(val)
            # Highlight red if bottleneck
            if val >= cap:
                bars[idx].set_color('r')
            else:
                bars[idx].set_color('b')
        ax.set_ylim(0, max([max(h) for h in history.values()])+5)
        ax.set_ylabel("Items in Buffer/Store")
        ax.set_title(f"Assembly Line Buffer Visualization (t={i})")

    ani = animation.FuncAnimation(fig, update, frames=len(next(iter(history.values()))), repeat=False, interval=200)
    plt.show()


In [6]:
item_input_sequence = [None, 'item1', None, 'item2', 'item3', None, 'item4', None, None, 'item5']


In [ ]:
[None, None, None, None, None, None, None, None, None, None]
['item1', None, None, None, None, None, None, None, None, None]
[None, 'item1', None, None, None, None, None, None, None, None]
[None, None, 'item1', None, None, None, None, None, None, None]
['item2', None, None, 'item1', None, None, None, None, None, None]
[None, 'item2', None, None, 'item1', None, None, None, None, None]
[None, None, 'item2', None, None, 'item1', None, None, None, None]
[None, None, None, 'item2', None, None, 'item1', None, None, None]
[None, None, None, None, 'item2', None, None, 'item1', None, None]
['item3', None, None, None, None, 'item2', None, None, 'item1', None]
[None, 'item3', None, None, None, None, 'item2', None, None, 'item1']
[None, None, 'item3', None, None, None, None, 'item2', None, 'item1']
[None, None, None, 'item3', None, None, None, None, 'item2', 'item1']
[None, None, None, None, 'item3', None, None, None, 'item2', 'item1']
['item4', None, None, None, None, 'item3', None, None, 'item2', 'item1']
[None, 'item4', None, None, None, None, 'item3', None, 'item2', 'item1']
[None, None, 'item4', None, None, None, None, 'item3', 'item2', 'item1']
[None, None, None, 'item4', None, None, None, 'item3', 'item2', 'item1']
Next machine cleared
[None, None, None, None, 'item4', None, None, None, 'item3', 'item2']
[None, None, None, None, None, 'item4', None, None, None, 'item3']
[None, None, None, None, None, None, 'item4', None, None, None]
[None, None, None, None, None, None, None, 'item4', None, None]
[None, None, None, None, None, None, None, None, 'item4', None]
[None, None, None, None, None, None, None, None, None, 'item4']
[None, None, None, None, None, None, None, None, None, None]

In [8]:
import simpy

class ConveyorBeltSim:
    def __init__(self, env, capacity, conveyor_time=1.0):
        self.env = env
        self.capacity = capacity
        self.conveyor = [None] * capacity
        self.conveyor_time = conveyor_time
        self.history = []
        self.next_machine_clear = False  # Manual control flag

    def run(self, item_source):
        while True:
            # 1. Manual "next machine" clears the rightmost slot when flag set
            if self.next_machine_clear and self.conveyor[-1] is not None:
                print(f"Time {self.env.now}: Next machine cleared {self.conveyor[-1]}")
                self.conveyor[-1] = None
                self.next_machine_clear = False

            # 2. Move items forward if space in front (right to left, skipping 0)
            for i in range(self.capacity - 1, 0, -1):
                if self.conveyor[i] is None and self.conveyor[i-1] is not None:
                    self.conveyor[i] = self.conveyor[i-1]
                    self.conveyor[i-1] = None

            # 3. Load new item at entry if possible
            if self.conveyor[0] is None and len(item_source.items) > 0:
                self.conveyor[0] = item_source.items.pop(0)
                print(f"Time {self.env.now}: Loaded {self.conveyor[0]}")

            print(list(self.conveyor))
            self.history.append(list(self.conveyor))
            yield self.env.timeout(self.conveyor_time)

class ItemSource(simpy.Store):
    pass

def item_feeder(env, source, items, interval):
    for item in items:
        yield env.timeout(interval)
        yield source.put(item)
        print(f"Time {env.now}: Item {item} added to source")

def manual_next_machine(env, conveyor, clear_steps):
    for t in clear_steps:
        yield env.timeout(t - env.now)
        conveyor.next_machine_clear = True

# if __name__ == "__main__":
env = simpy.Environment()
items = ['item1', 'item2', 'item3', 'item4']
item_source = ItemSource(env)
conveyor = ConveyorBeltSim(env, capacity=10, conveyor_time=1)
env.process(item_feeder(env, item_source, items, interval=2))

# Manually clear the next machine at these simulation times:
clear_steps = [17,20,25]  # Try delaying clear to see items back up behind a stuck item

env.process(conveyor.run(item_source))
env.process(manual_next_machine(env, conveyor, clear_steps))

env.run(until=30)

print("\nConveyor history (left is entry, right is exit):")
for i, state in enumerate(conveyor.history):
    print("time:",i, " state:",state)


[None, None, None, None, None, None, None, None, None, None]
[None, None, None, None, None, None, None, None, None, None]
Time 2: Loaded item1
['item1', None, None, None, None, None, None, None, None, None]
Time 2: Item item1 added to source
[None, 'item1', None, None, None, None, None, None, None, None]
Time 4: Loaded item2
['item2', None, 'item1', None, None, None, None, None, None, None]
Time 4: Item item2 added to source
[None, 'item2', None, 'item1', None, None, None, None, None, None]
Time 6: Loaded item3
['item3', None, 'item2', None, 'item1', None, None, None, None, None]
Time 6: Item item3 added to source
[None, 'item3', None, 'item2', None, 'item1', None, None, None, None]
Time 8: Loaded item4
['item4', None, 'item3', None, 'item2', None, 'item1', None, None, None]
Time 8: Item item4 added to source
[None, 'item4', None, 'item3', None, 'item2', None, 'item1', None, None]
[None, None, 'item4', None, 'item3', None, 'item2', None, 'item1', None]
[None, None, None, 'item4', None,

In [10]:
import simpy

class ConveyorBelt:
    def __init__(self, env, name, capacity, input_buffer, output_buffer, conveyor_time):
        self.env = env
        self.name = name
        self.capacity = capacity
        self.input_buffer = input_buffer  # Should have .items (like SimPy Store) or .get()
        self.output_buffer = output_buffer  # Should have .items or .put()
        self.conveyor = [None] * capacity
        self.conveyor_time = conveyor_time

        self.history = []

    def move_conveyor(self):
        """Move items right if possible (no wrapping, physical behavior)."""
        # 1. Try to unload rightmost slot into output buffer if possible
        right_item = self.conveyor[-1]
        output_has_space = (
            hasattr(self.output_buffer, "items") and
            len(self.output_buffer.items) < getattr(self.output_buffer, "capacity", float('inf'))
        )
        if right_item is not None and output_has_space:
            # Use .put() if output_buffer is SimPy Store, else just append for list
            if hasattr(self.output_buffer, "put"):
                self.env.process(self.output_buffer.put(right_item))
            else:
                self.output_buffer.items.append(right_item)
            print(f"Time {self.env.now}: Output {right_item} from {self.name}")
            self.conveyor[-1] = None

        # 2. Move items forward (right to left, skipping index 0)
        for i in range(self.capacity - 1, 0, -1):
            if self.conveyor[i] is None and self.conveyor[i-1] is not None:
                self.conveyor[i] = self.conveyor[i-1]
                self.conveyor[i-1] = None

        # 3. Try to load new item at the entry (slot 0) if possible
        if self.conveyor[0] is None and hasattr(self.input_buffer, "items") and len(self.input_buffer.items) > 0:
            item = self.input_buffer.items.pop(0)
            self.conveyor[0] = item
            print(f"Time {self.env.now}: Loaded {item} onto {self.name}")

    def run(self):
        while True:
            self.move_conveyor()
            # (Optional) Log state for visualization
            self.history.append(list(self.conveyor))
            print(f"Time {self.env.now}: {self.conveyor}")
            yield self.env.timeout(self.conveyor_time)

    def process(self):
        self.proc = self.env.process(self.run())


class Buffer(simpy.Store):
    # Inherit for input/output if you want true SimPy behavior.
    pass

env = simpy.Environment()
in_buffer = Buffer(env, capacity=100)
out_buffer = Buffer(env, capacity=100)
# Pre-load input buffer with items
for i in range(10):
    in_buffer.items.append(f"item_{i}")

conveyor = ConveyorBelt(env, "Conveyor1", capacity=10, input_buffer=in_buffer, output_buffer=out_buffer, conveyor_time=1.0)
conveyor.process()

env.run(until=20)

Time 0: Loaded item_0 onto Conveyor1
Time 0: ['item_0', None, None, None, None, None, None, None, None, None]
Time 1.0: Loaded item_1 onto Conveyor1
Time 1.0: ['item_1', 'item_0', None, None, None, None, None, None, None, None]
Time 2.0: Loaded item_2 onto Conveyor1
Time 2.0: ['item_2', 'item_1', 'item_0', None, None, None, None, None, None, None]
Time 3.0: Loaded item_3 onto Conveyor1
Time 3.0: ['item_3', 'item_2', 'item_1', 'item_0', None, None, None, None, None, None]
Time 4.0: Loaded item_4 onto Conveyor1
Time 4.0: ['item_4', 'item_3', 'item_2', 'item_1', 'item_0', None, None, None, None, None]
Time 5.0: Loaded item_5 onto Conveyor1
Time 5.0: ['item_5', 'item_4', 'item_3', 'item_2', 'item_1', 'item_0', None, None, None, None]
Time 6.0: Loaded item_6 onto Conveyor1
Time 6.0: ['item_6', 'item_5', 'item_4', 'item_3', 'item_2', 'item_1', 'item_0', None, None, None]
Time 7.0: Loaded item_7 onto Conveyor1
Time 7.0: ['item_7', 'item_6', 'item_5', 'item_4', 'item_3', 'item_2', 'item_1', 'i

ValueError: <StorePut() object at 0x7b299a904f40> is not a generator.